<a href="https://colab.research.google.com/github/Matiiwtcp/Proyecto_final_bigdata/blob/main/Proyecto_final_MLBG.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [223]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [224]:
import pandas as pd
import geopandas as gpd
import numpy as np
import matplotlib.pyplot as plt
import plotly.express as px
import pandas as pd
import geopandas as gpd
import numpy as np
import matplotlib.pyplot as plt
import plotly.express as px
import plotly.graph_objects as go
from plotly.subplots import make_subplots

#WIKIPEDIA
#PBI, Esperanza de vida (2020) : https://www.kaggle.com/datasets/daniboy370/world-data-by-country-2020

#Indice publicado por el Instituto Fraser mide el grado en que las políticas e instituciones de los países apoyan la libertad económica.
#Libertad Economica: (2018) https://www.kaggle.com/datasets/gsutters/economic-freedom/data

#Tasa de finalización - Base de datos mundial de UNICEF (abril de 2021)
#Tasa de niños y jóvenes sin escolarizar - Instituto de Estadística de la UNESCO (marzo de 2021).
#Tasa de NO Escolaridad y de Finalizacion https://www.kaggle.com/datasets/nelgiriyewithana/world-educational-data


In [225]:
# Principal medida mundial de la libertad económica, clasificando a los países en función de cinco áreas:
# Área 1: Tamaño del gobierno - A medida que aumentan el gasto público, los impuestos y el tamaño de las empresas controladas por el gobierno,
# la toma de decisiones del gobierno es sustituida por la elección individual y la libertad económica se reduce.

# Área 2: Sistema legal y derechos de propiedad - La protección de las personas y sus bienes legítimamente adquiridos es un elemento central tanto de
# la libertad económica como de la sociedad civil. De hecho, es la función más importante del gobierno.

# Área 3: Dinero sólido - La inflación erosiona el valor de los salarios y ahorros ganados legítimamente.
# Por lo tanto, el dinero sólido es esencial para proteger los derechos de propiedad. Cuando la inflación no solo es alta, sino también volátil,
# se vuelve difícil para las personas planificar el futuro y, por lo tanto, utilizar la libertad económica de manera efectiva.

# Área 4: Libertad para comerciar internacionalmente - La libertad de intercambio, en su sentido más amplio, comprar, vender, hacer contratos, etc.,
# es esencial para la libertad económica, que se reduce cuando la libertad de intercambio no incluye a las empresas y a los individuos de
# otras naciones.

# Área 5: Regulación - Los gobiernos no solo utilizan una serie de herramientas para limitar el derecho a intercambiar internacionalmente,
# sino que también pueden desarrollar regulaciones onerosas que limitan el derecho a intercambiar, obtener crédito, contratar o trabajar para
# quien usted desee, o operar libremente su negocio.


ruta_de_archivos = '/content/drive/MyDrive/libertadeconomica.csv'
dataframeLibertadEconomica = pd.read_csv(ruta_de_archivos, encoding='latin1')
df_le = dataframeLibertadEconomica.copy()

# Convertir valores nulos a NaN
df_le.replace('Datos nulos', np.nan, inplace=True)

# Filtrar las filas donde 'year' sea igual a 2016
df_le = df_le[df_le['year'] == 2016]


# Seleccionar las columnas 'ECONOMIC FREEDOM' y 'countries'
df_le = df_le[['ECONOMIC FREEDOM', 'countries']]

# Eliminar filas con NaN
df_le = df_le.dropna()

In [226]:
ruta_de_archivos2 = '/content/drive/MyDrive/Global_Education.csv'
dataframeEducacionGlobal = pd.read_csv(ruta_de_archivos2, encoding='latin1')
df_eg = dataframeEducacionGlobal.copy ()
# Convertir valores nulos a NaN
df_eg.replace('Datos nulos', np.nan, inplace=True)
# Eliminar filas con NaN
df_eg = df_eg.dropna()

# Ajustar nombres de columnas eliminando espacios adicionales
df_eg.columns = df_eg.columns.str.strip()


# Diccionario de traducción de nombres de columnas
nuevos_nombres = {
    'Countries and areas': 'Paises y Zonas',
    'Latitude': 'Latitud',
    'Longitude': 'Longitud',
    'OOSR_Pre0Primary_Age_Male': 'Tasa No Escolarización Preescolar (Varones)',
    'OOSR_Pre0Primary_Age_Female': 'Tasa No Escolarización Preescolar (Mujeres)',
    'OOSR_Primary_Age_Male': 'Tasa No Escolarización Primaria (Varones)',
    'OOSR_Primary_Age_Female': 'Tasa No Escolarización Primaria (Mujeres)',
    'OOSR_Lower_Secondary_Age_Male': 'Tasa No Escolarización Primer Ciclo Secundaria (Varones)',
    'OOSR_Lower_Secondary_Age_Female': 'Tasa No Escolarización Primer Ciclo Secundaria (Mujeres)',
    'OOSR_Upper_Secondary_Age_Male': 'Tasa No Escolarización Segundo Ciclo Secundaria (Varones)',
    'OOSR_Upper_Secondary_Age_Female': 'Tasa No Escolarización Segundo Ciclo Secundaria (Mujeres)',
    'Completion_Rate_Primary_Male': 'Tasa Finalización Educación Primaria (Varones)',
    'Completion_Rate_Primary_Female': 'Tasa Finalización Educación Primaria (Mujeres)',
    'Completion_Rate_Lower_Secondary_Male': 'Tasa Finalización Primer Ciclo Secundaria (Varones)',
    'Completion_Rate_Lower_Secondary_Female': 'Tasa Finalización Primer Ciclo Secundaria (Mujeres)',
    'Completion_Rate_Upper_Secondary_Male': 'Tasa Finalización Segundo Ciclo Secundaria (Varones)',
    'Completion_Rate_Upper_Secondary_Female': 'Tasa Finalización Segundo Ciclo Secundaria (Mujeres)',
    'Grade_2_3_Proficiency_Reading': 'Competencia Lectura 2º y 3º Grado',
    'Grade_2_3_Proficiency_Math': 'Competencia Matemáticas 2º y 3º Grado',
    'Primary_End_Proficiency_Reading': 'Competencia Lectora Final Educación Primaria',
    'Primary_End_Proficiency_Math': 'Dominio Matemáticas Final Educación Primaria',
    'Lower_Secondary_End_Proficiency_Reading': 'Competencia Lectora Final Primer Ciclo Secundaria',
    'Lower_Secondary_End_Proficiency_Math': 'Dominio Matemáticas Final Primer Ciclo Secundaria',
    'Youth_15_24_Literacy_Rate_Male': 'Tasa Alfabetización Jóvenes (Varones 15-24 años)',
    'Youth_15_24_Literacy_Rate_Female': 'Tasa Alfabetización Jóvenes (Mujeres 15-24 años)',
    'Birth_Rate': 'Tasa Natalidad',
    'Gross_Primary_Education_Enrollment': 'Matrícula Bruta Educación Primaria',
    'Gross_Tertiary_Education_Enrollment': 'Matrícula Bruta Educación Terciaria',
    'Unemployment_Rate': 'Tasa Desempleo'
}


# Renombrar las columnas en el DataFrame
df_eg.rename(columns=nuevos_nombres, inplace=True)

In [227]:
#Una lista de países del mundo por producto interno bruto (en paridad de poder adquisitivo) per cápita,
#es decir, el valor de paridad de poder adquisitivo (PPA) de todos los bienes y servicios finales producidos dentro de un país en un año determinado.

ruta_de_archivos3 = '/content/drive/MyDrive/PBIpercapita.csv'
dataframePBIpercapita = pd.read_csv(ruta_de_archivos3, encoding='latin1')
df_pbi = dataframePBIpercapita .copy()

# Convertir valores nulos a NaN
df_pbi.replace('Datos nulos', np.nan, inplace=True)

# Eliminar filas con NaN
df_pbi= df_pbi.dropna()


In [228]:
#La esperanza de vida es una medida estadística del tiempo promedio (ver más abajo) que se espera que viva un organismo

ruta_de_archivos4 = '/content/drive/MyDrive/Lifeexpectancy.csv'
dataframeEsperanzaVida = pd.read_csv(ruta_de_archivos4, encoding='latin1')
df_ev = dataframeEsperanzaVida .copy()

# Convertir valores nulos a NaN
df_ev.replace('Datos nulos', np.nan, inplace=True)

# Eliminar filas con NaN
df_ev= df_ev.dropna()



In [229]:
#Esta lista de países contiene la tasa de suicidios según los datos publicados por la Organización Mundial de la Salud (OMS).
ruta_de_archivos5 = '/content/drive/MyDrive/Suicide rate.csv'
dataframeSuicidios = pd.read_csv(ruta_de_archivos5, encoding='latin1')
df_s = dataframeSuicidios.copy()

# Convertir valores nulos a NaN
df_s.replace('Datos nulos', np.nan, inplace=True)

# Eliminar filas con NaN
df_s= df_s.dropna()


In [230]:
#funcion para graficar
def create_choropleth(df, location_column, z_column, text_column, title, colorbar_title):
    data = go.Choropleth(
        locations=df[location_column],
        locationmode='country names',
        z=df[z_column],
        text=df[text_column],
        colorscale='Viridis',
        colorbar={'title': colorbar_title}
    )

    fig = go.Figure(data=[data])
    fig.update_geos(projection_type='natural earth')
    fig.update_layout(
        title_text=title,
        geo=dict(showframe=False, coastlinecolor='white', showcoastlines=True)
    )

    return fig

#funcion para graficar varios
def create_multi_choropleth(df, location_column, z_column, text_column, title, colorbar_title, zmin=None, zmax=None):
    data = go.Choropleth(
        locations=df[location_column],
        locationmode='country names',
        z=df[z_column],
        text=df[text_column],
        colorscale='Viridis',
        colorbar={'title': colorbar_title},
        zmin=zmin,
        zmax=zmax
    )

    return data





In [231]:
# Economic Freedom
fig_economic_freedom = create_choropleth(df_le, 'countries', 'ECONOMIC FREEDOM', 'countries', 'Mapa de Libertad Económica', 'Libertad Economica')
fig_economic_freedom.show()


In [232]:
#Life Expectancy
fig_life_expectancy = create_choropleth(df_ev, 'Country', 'Life expectancy', 'Country', 'Mapa de Esperanza de vida', 'Esperanza de vida')
fig_life_expectancy.show()


In [233]:
#PBI per Capita
fig_pbi_per_capita = create_choropleth(df_pbi, 'Country', 'GDP per capita', 'Country', 'Mapa de PBI', 'PBI Per Capita')
fig_pbi_per_capita.show()

In [234]:
#Suicidios
fig_suicidios = create_choropleth(df_s, 'Country', 'Suicide rate', 'Country', 'Mapa de Ratio de Suicidios', 'Suicidios cada 100mil habitantes')
fig_suicidios.show()

In [235]:

# Crear subgráficos con tasas de no escolarización por nivel y género
niveles_educacion = ['Preescolar', 'Primaria', 'Primer Ciclo Secundaria', 'Segundo Ciclo Secundaria']
num_filas = 2

fig = make_subplots(rows=num_filas, cols=len(niveles_educacion) // num_filas,
                    subplot_titles=[f'{nivel} ' for nivel in niveles_educacion],
                    specs=[[{'type': 'choropleth'}] * (len(niveles_educacion) // num_filas) for _ in range(num_filas)])

colorbar_title = "Tasa de No Escolarización"
zmin = 0  # Valor mínimo para la barra de colores
zmax = 100  # Valor máximo para la barra de colores

for i, nivel in enumerate(niveles_educacion, start=1):
    columna_varones = f'Tasa No Escolarización {nivel} (Varones)'
    columna_mujeres = f'Tasa No Escolarización {nivel} (Mujeres)'
    df_eg[f'Tasa No Escolarización {nivel} (Media)'] = df_eg[[columna_varones, columna_mujeres]].mean(axis=1)

    df_filtered = df_eg[df_eg[f'Tasa No Escolarización {nivel} (Media)'] > 0]
    df_sorted = df_filtered.sort_values(by=f'Tasa No Escolarización {nivel} (Media)', ascending=False)

    if not df_sorted.empty:
        data = create_multi_choropleth(df_sorted, 'Paises y Zonas', f'Tasa No Escolarización {nivel} (Media)',
                                  'Paises y Zonas', f'Tasas de No Escolarización {nivel}', colorbar_title, zmin, zmax, )

        row = (i - 1) % num_filas + 1
        col = (i - 1) // num_filas + 1
        fig.add_trace(data, row=row, col=col)

fig.update_geos(projection_type='natural earth')
fig.update_layout(
    height=800,
    width=1400,
    title_text="Tasas de No Escolarización por Nivel"
)
fig.show()

In [236]:
# Crear subgráficos con tasas de finalización por nivel y género
niveles_educacion = ['Educación Primaria', 'Primer Ciclo Secundaria', 'Segundo Ciclo Secundaria']
num_filas = 3

fig = make_subplots(rows=num_filas, cols=len(niveles_educacion) // num_filas,
                    subplot_titles=[f'{nivel} - Media de Géneros' for nivel in niveles_educacion],
                    specs=[[{'type': 'choropleth'}] * (len(niveles_educacion) // num_filas) for _ in range(num_filas)])

colorbar_title = "Tasa Finalización"
zmin = 0  # Valor mínimo para la barra de colores
zmax = 100  # Valor máximo para la barra de colores

for i, nivel in enumerate(niveles_educacion, start=1):
    columna_varones = f'Tasa Finalización {nivel} (Varones)'
    columna_mujeres = f'Tasa Finalización {nivel} (Mujeres)'
    df_eg[f'Tasa Finalización {nivel} (Media)'] = df_eg[[columna_varones, columna_mujeres]].mean(axis=1)

    df_filtered = df_eg[df_eg[f'Tasa Finalización {nivel} (Media)'] > 0]
    df_sorted = df_filtered.sort_values(by=f'Tasa Finalización {nivel} (Media)', ascending=False)

    if not df_sorted.empty:
        data = create_multi_choropleth(df_sorted, 'Paises y Zonas', f'Tasa Finalización {nivel} (Media)',
                                  'Paises y Zonas', f'Tasas de Finalización {nivel}', colorbar_title, zmin, zmax)

        row = (i - 1) % num_filas + 1
        col = ((i - 1) // num_filas) % (len(niveles_educacion) // num_filas) + 1
        fig.add_trace(data, row=row, col=col)

fig.update_geos(projection_type='natural earth')
fig.update_layout(
    height=1000,
    width=1200,
    title_text="Tasas de Finalización por Nivel"
)
fig.show()